# Unykorn Ignite Notebook

Bridge Codespaces secrets → restart API → verify Xumm → apply Cloudflare DNS.

Note: If your Codespaces secrets show ‘0 repositories’, they won’t be injected. Either:
- Edit each secret (XAMANAPI, XAMANKEY, CLOUDFAREWORKERAPI / CLOUDFLARE_API_TOKEN) to be available to this repository (or All repositories), then restart the Codespace; or
- Manually export them in the shell (Step 1b) once per session.

## 0) Prereqs: Codespaces Secrets
- GitHub → Settings → Codespaces → Secrets.
- Ensure these exist and are assigned to this repo (not ‘0 repositories’):
  - `XAMANAPI`, `XAMANKEY` (Xumm)
  - `CLOUDFAREWORKERAPI` or `CLOUDFLARE_API_TOKEN` (Cloudflare)
- Optional: `CLOUDFLARE_ZONE_ID` (script can resolve from domain).
- Restart the Codespace so secrets are injected into the container.

## 1a) Bridge and inspect environment
Maps common secret names into what the repo expects and shows a masked summary.

In [ ]:
set -e
source scripts/env-bridge.sh
echo "== masked env summary =="
env | egrep '^(XUMM_API_|XAMANAPI|XAMANKEY|CF_API_TOKEN|CLOUDFLARE_API_TOKEN|CLOUDFAREWORKERAPI|CF_ZONE_ID|CLOUDFLARE_ZONE_ID|DOMAIN)=' | sed -E 's/=.*/=***masked***/' || true

## 1b) Manual shim (only if secrets didn’t inject)
Map your Codespaces secrets into the expected env vars for this session, then re-run 1a.

In [ ]:
# Manual shim (edit if you prefer to paste literal values)
export XUMM_API_KEY="$XAMANAPI"
export XUMM_API_SECRET="$XAMANKEY"
export CF_API_TOKEN="$CLOUDFAREWORKERAPI"   # or: export CF_API_TOKEN="$CLOUDFLARE_API_TOKEN"
export DOMAIN="unykorn.org"
source scripts/env-bridge.sh
echo "== masked (post-shim) =="
env | egrep '^(XUMM_API_|CF_API_TOKEN|DOMAIN)=' | sed -E 's/=.*/=***masked***/'

## 2) Restart API and verify Xumm
Kills any stuck process on port 4000, relaunches API, then checks /health and /xumm/ping.

In [ ]:
npm run api:reset
echo "== /health =="
curl -sS http://localhost:4000/health | jq
echo "== /xumm/ping =="
curl -sS http://localhost:4000/xumm/ping | jq || true

## 3) Cloudflare DNS automation
Requires `CF_API_TOKEN` in the environment (or `CLOUDFLARE_API_TOKEN` / `CLOUDFAREWORKERAPI`).
Update `--target` to your actual API host (e.g., `api.fly.dev`, `onrender.com`, etc.).

In [ ]:
echo "== Cloudflare: verify zone =="
npm run cf:verify -- --domain "$DOMAIN"
echo "== Cloudflare: Pages on apex + www =="
npm run cf:pages:apex -- --domain "$DOMAIN" --proxied true
echo "== Cloudflare: set API CNAME =="
# Replace target below with your actual API host
npm run cf:set:api -- --domain "$DOMAIN" --target api.fly.dev --proxied true
echo "== Cloudflare: show records =="
npm run cf:show -- --domain "$DOMAIN"

## 4) Troubleshooting
- EADDRINUSE (port busy): `npm run api:stop` then `npm run api:start`.
- Missing Xumm keys: export in the same terminal as the API, then `npm run api:reset`.
- GitHub Codespaces forwarded URL not resolving: Ports tab → 4000 → Visibility: Public → Open in Browser.
- CORS in production: set `CORS_ORIGINS=https://unykorn.org,https://www.unykorn.org,https://trustiva.io,https://www.trustiva.io` on the API host.
- GitHub Pages repo Variables (Actions → Variables):
  - `NEXT_PUBLIC_API_BASE=https://api.unykorn.org`
  - `NEXT_PUBLIC_BRAND=Unykorn`
  - `NEXT_PUBLIC_NETWORK_LABEL=Mainnet`
  - `NEXT_PUBLIC_HIDE_CHECK_NAV=false`
  - `PAGES_CNAME=unykorn.org` (optional override; `apps/web/public/CNAME` already set)

In [ ]:
echo "✅ Unykorn Ignite complete: API + Xumm + Cloudflare all verified $(date)"